In [68]:
import numpy as np
from quaos.core.paulis import PauliSum, PauliString

In [ ]:
# -------------------------
# Qubits and positions
# -------------------------
def list_qubits(Nx: int, 
                Ny: int, 
                periodic: bool = True
                ) -> list[tuple[str, tuple[int, int]]]:
    """
    List all qubits (edges) on an Nx-by-Ny lattice.
    Each qubit is denoted ('h', i, j) for a horizontal edge (i->i+1 at row j) 
    or ('v', i, j) for a vertical edge (i, j->j+1).  Periodic=True wraps around.
    """
    qubits = []
    # Horizontal edges
    for j in range(Ny):
        max_i = Nx if periodic else Nx-1
        for i in range(max_i):
            qubits.append(('h', i, j))
    # Vertical edges
    for i in range(Nx):
        max_j = Ny if periodic else Ny-1
        for j in range(max_j):
            qubits.append(('v', i, j))
    # Check
    if periodic:
        assert len(qubits) == 2 * Nx * Ny, "Periodic qubit count mismatch"
    elif not periodic:
        assert len(qubits) == Nx * (Ny-1) + Ny * (Nx-1), "Non-periodic qubit count mismatch"
    else:
        raise ValueError("Invalid boundary condition")
    return qubits

In [ ]:
# -------------------------
# Star operators (Z-type)
# -------------------------
def build_star_ops(Nx: int, 
                   Ny: int, periodic: bool = True
                   ) -> list[list[int]]:
    """
    Build the list of star operators.  Each star is a list of edge-indices 
    (from list_qubits) on which Z acts.
    """
    qubits = list_qubits(Nx, Ny, periodic)
    index_of = {q: idx for idx,q in enumerate(qubits)}
    star_ops = []
    for i in range(Nx):
        for j in range(Ny):
            edges = []
            # Horizontal edge to the right of (i,j)
            if (i < Nx-1) or periodic:
                qi = ('h', i, j)
                if qi in index_of:
                    edges.append(index_of[qi])
            # Horizontal edge to the left of (i,j)
            i_left = (i-1) % Nx if periodic else i-1
            if (i > 0) or periodic:
                qi = ('h', i_left, j)
                if qi in index_of:
                    edges.append(index_of[qi])
            # Vertical edge above (i,j)
            if (j < Ny-1) or periodic:
                qi = ('v', i, j)
                if qi in index_of:
                    edges.append(index_of[qi])
            # Vertical edge below (i,j)
            j_below = (j-1) % Ny if periodic else j-1
            if (j > 0) or periodic:
                qi = ('v', i, j_below)
                if qi in index_of:
                    edges.append(index_of[qi])
            if edges:
                star_ops.append(sorted(set(edges)))
    # Check
    assert len(star_ops) == Nx * Ny, "Star operator count mismatch"
    return star_ops


In [ ]:
# -------------------------
# Plaquette operators (X-type)
# -------------------------
def build_plaquette_ops(Nx: int, 
                        Ny: int, 
                        periodic: bool = True
                        ) -> list[list[int]]:
    """
    Build the list of plaquette operators.  Each plaquette is a list of 
    edge-indices (from list_qubits) on which X acts.
    """
    qubits = list_qubits(Nx, Ny, periodic)
    index_of = {q: idx for idx,q in enumerate(qubits)}
    plaquette_ops = []
    max_i = Nx if periodic else Nx-1
    max_j = Ny if periodic else Ny-1
    for i in range(max_i):
        for j in range(max_j):
            edges = []
            # Bottom horizontal edge of plaquette at (i,j)
            if True:
                qi = ('h', i, j)
                if qi in index_of:
                    edges.append(index_of[qi])
                elif periodic and i == Nx-1:  # wrap around right
                    edges.append(index_of[('h', i, j)])
            # Top horizontal edge
            top_j = (j+1) % Ny if periodic else j+1
            if top_j < Ny:
                qi = ('h', i, top_j)
                if qi in index_of:
                    edges.append(index_of[qi])
                elif periodic and i == Nx-1:
                    edges.append(index_of[('h', i, top_j)])
            # Left vertical edge
            qi = ('v', i, j)
            if qi in index_of:
                edges.append(index_of[qi])
            elif periodic and j == Ny-1:
                edges.append(index_of[('v', i, j)])
            # Right vertical edge
            right_i = (i+1) % Nx if periodic else i+1
            if right_i < Nx:
                qi = ('v', right_i, j)
                if qi in index_of:
                    edges.append(index_of[qi])
                elif periodic and j == Ny-1:
                    edges.append(index_of[('v', right_i, j)])
            # Skip incomplete plaquettes on open boundary
            if not periodic and (i == Nx-1 or j == Ny-1):
                continue
            if edges:
                plaquette_ops.append(sorted(set(edges)))
    # Check
    if periodic:
        assert len(plaquette_ops) == Nx * Ny, "Periodic plaquette count mismatch"
    elif not periodic:
        assert len(plaquette_ops) == (Nx-1) * (Ny-1), "Non-periodic plaquette count mismatch"
    else:
        raise ValueError("Invalid boundary condition")
    return plaquette_ops


In [34]:
# -------------------------
# Gauge terms
# -------------------------
def build_gauge_ops(Nx: int, 
                Ny: int, 
                periodic: bool = True
                ) -> list[list[int]]:
    """
    Build the list of gauge operators.  Each gauge operator consists of a single Z operator acting on a qubit.
    """
    qubits = list_qubits(Nx, Ny, periodic)
    gauge_ops = []
    for idx,q in enumerate(qubits):
        gauge_ops.append([idx])
    return gauge_ops

In [47]:
# -------------------------
# Toric code Hamiltonian
# -------------------------
def toric_code_hamiltonian(c_x: float, 
                           c_z: float,
                           c_g: float,
                           Nx: int, 
                           Ny: int, 
                           periodic: bool = True
                           ) -> tuple[list[str], list[float]]:
    """
    Construct toric code Hamiltonian terms for given Nx, Ny, boundary, and coefficients.
    Returns (terms, coeffs), where each term is a string of 'xNzM' tokens for each qubit.
    """
    qubits = list_qubits(Nx, Ny, periodic)
    Nq = len(qubits)
    stars = build_star_ops(Nx, Ny, periodic)
    plaquettes = build_plaquette_ops(Nx, Ny, periodic)
    gauges = build_gauge_ops(Nx, Ny, periodic)
    terms = []
    coeffs = []
    # Star terms (Z on each edge in the star)
    if abs(c_z) > 10**-12:
        for edge_list in stars:
            word = []
            for q in range(Nq):
                if q in edge_list:
                    word.append('x0z1')  # Z on this qubit
                else:
                    word.append('x0z0')  # identity
            terms.append(' '.join(word))
            coeffs.append(c_z)
    # Plaquette terms (X on each edge in the plaquette)
    if abs(c_x) > 10**-12:
        for edge_list in plaquettes:
            word = []
            for q in range(Nq):
                if q in edge_list:
                    word.append('x1z0')  # X on this qubit
                else:
                    word.append('x0z0')  # identity
            terms.append(' '.join(word))
            coeffs.append(c_x)
    # Gauge terms (Z on each edge) - only if c_g != 0
    if abs(c_g) > 10**-12:
        for edge_list in gauges:
            word = []
            for q in range(Nq):
                if q in edge_list:
                    word.append('x0z1')  # Z on this qubit
                else:
                    word.append('x0z0')  # identity
            terms.append(' '.join(word))
            coeffs.append(c_g)
    return terms, coeffs

# Example

In [150]:
Nx = 2
Ny = 2

periodic = True

n_qubits = 2* Nx * Ny if periodic else Nx * (Ny - 1) + Ny * (Nx - 1)

c_x = 1.
c_z = 2.
c_g = 0.

In [151]:
# q_list = list_qubits(Nx, Ny, periodic)
# q_list

In [152]:
# s_op = build_star_ops(Nx, Ny, periodic)
# s_op

In [153]:
# p_op = build_plaquette_ops(Nx, Ny, periodic)
# p_op

In [154]:
ps, cc = toric_code_hamiltonian(c_x, c_z, c_g, Nx, Ny, periodic)

In [ ]:
# TODO: Understand why parameter cc is highlighted to be wrong...
h = PauliSum(ps, weights=cc, dimensions=[2 for _ in range(n_qubits)])

In [147]:
h.matrix_form()

<Compressed Sparse Row sparse matrix of dtype 'complex128'
	with 1256 stored elements and shape (256, 256)>

In [157]:
# TODO: (possibly already done) - include this function in the relevant .py file in core
def ground_state_TMP(P: PauliSum, 
                     only_gs: bool = True
                     ) -> tuple[np.ndarray, np.ndarray]:
    """
    Compute eigenvalues/eigenvectors of `P` and (by default) pick
    the ground-state energy (`only_gs` = `True`).

    Parameters
    ----------
    P : PauliSum
        A PauliSum object representing the Hamiltonian/operator.
    only_gs : bool, optional
        If `True` (default), only the ground-state (energy) is kept in `gs` (`en`).
         If `False`, all eigenvectors (eigenvalues) are kept in `gs` (`en`).

    Returns
    -------
    en : float or numpy.ndarray
        If `only_gs` is `True`, the lowest eigenvalue (ground-state energy).
        Otherwise, a 1D array of all eigenvalues sorted ascending.
    gs : numpy.ndarray
        Eigenvectors sorted to match ``en``.

    Raises
    ------
    AssertionError
        If the (internally normalized) eigenvector(s) do not yield real
        expectation values matching the corresponding eigenvalue(s) within
        ``1e-10``.

    Notes
    -----
    - Since :func:`numpy.linalg.eig` is used, the input matrix is treated as
      general (not assumed Hermitian). If the operator is Hermitian, consider
      using :func:`numpy.linalg.eigh` for improved numerical stability.
    """
    # Convert PauliSum to matrix form
    m = P.matrix_form()
    m = m.toarray()
    # Get eigenvalues and eigenvectors
    val, vec = np.linalg.eig(m)
    val = np.real(val)
    vec = np.transpose(vec)
    # Ordering
    tmp_index = np.argsort(val)
    en = val[tmp_index]
    gs = vec[tmp_index]
    # Prepare output
    if only_gs:
        en = en[0]
        gs_out = gs[0]
        gs_out = np.transpose(gs_out)
        gs_out = gs_out / np.linalg.norm(gs_out)
    else:
        gs_out = []
        for el in gs:
            el = np.transpose(el)
            el = el / np.linalg.norm(el)
            gs_out.append(el)
        gs_out = np.array(gs_out)
    # Checks
    exp_en = []
    if not only_gs:
        for el in gs_out:
            exp_en.append(np.transpose(np.conjugate(el)) @ m @ el)
        exp_en = np.array(exp_en)
    else:
        exp_en = np.transpose(np.conjugate(gs_out)) @ m @ gs_out
    assert np.max(abs(en - exp_en)) < 10**-10, "The ground state does not yield a real value <gs | H |gs> = {}".format(exp_en)
    # Return
    return en, gs


In [158]:
vals, vecs = ground_state_TMP(h, only_gs=True)

vals

np.float64(-12.000000000000053)